In [26]:
import numpy as np

# mass of solar system bodies in kg
body_mass = {'Sun': 1.989e30, 'Venus': 4.87e24, 'Earth': 5.97e24, 
             'Mars': 0.642e24, 'Jupiter': 1898e24, 'Saturn': 568e24, 
             'Titan': 0.1345e24}

# distance from sun in km
distance = {'Venus': 108.2e6, 'Earth': 149.6e6, 'Mars': 227.9e6,
             'Jupiter': 778.6e6, 'Saturn': 1433.5e6}

# initial radial degree 
initial_deg = {'Venus': 14.22, 'Earth': 219.09, 'Mars': 229.93, 
                'Jupiter': 172.26, 'Saturn': 252.77}

# orbital velocities in km/day
orbital_velocity = {'Venus': 1.9414e7, 'Earth': 3.15533e7, 'Mars': 5.93568e7, 
                   'Jupiter': 7.05542e7, 'Saturn': 1.308528e8}

def polar_to_cartesian(initial_deg, distance):
    """converts polar coordinates to cartesian coordinates
    Input:
    ------
    initial_deg
        initial angle in degrees
    distance
        distance from sun to planet (used for conversion calculation)
        
    Output:
    -------
    x, y
        x and y coordinates where sun is at origin
    """
    rad = initial_deg*(np.pi/180)
    x = distance*np.cos(rad)
    y = distance*np.sin(rad)
    return np.array([x, y])



In [27]:
initial_positions = polar_to_cartesian(initial_deg['Venus'], distance['Venus'])
print(initial_positions)

[  1.04884715e+08   2.65788724e+07]
